### move primitives

In [ ]:
import os
import tensorflow as tf
import numpy as np
import json
from tqdm import tqdm


os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# ==== Path ====
tfrecord_dir = "/data/gck/vla_planning/dataset/dexart/bucket_dex_art_dataset/1.0.0"
save_dir = "./results_move_primitives/bucket"
os.makedirs(save_dir, exist_ok=True)

# ==== tfrecord ====
tfrecord_files = sorted([
    os.path.join(tfrecord_dir, f)
    for f in os.listdir(tfrecord_dir)
    if f.startswith("dex_art_dataset-train.tfrecord")
])

# ==== move description  ====
def describe_move(move_vec):
    direction_names = [
        {-1: "backward", 0: None, 1: "forward"},   # x
        {-1: "right",    0: None, 1: "left"},      # y
        {-1: "down",     0: None, 1: "up"},        # z
    ]

    move_descriptions = [direction_names[i][move_vec[i]] for i in range(3)]
    move_descriptions = [desc for desc in move_descriptions if desc is not None]

    if len(move_descriptions) == 0:
        return "stop"
    else:
        return "move " + " ".join(move_descriptions)


# ==== classify move ====
def classify_movement(move, threshold=0.003):
    diff = move[-1] - move[0]

    if np.sum(np.abs(diff[:3])) > 3 * threshold:
        diff[:3] *= 3 * threshold / np.sum(np.abs(diff[:3]))

    move_vec = 1 * (diff[:3] > threshold) - 1 * (diff[:3] < -threshold)
    return describe_move(move_vec), move_vec


# ==== get all move primitives from a sequence ====
def get_move_primitives_from_states(states):
    move_trajs = [states[i : i + 4] for i in range(len(states) - 1)]
    primitives = [classify_movement(move) for move in move_trajs]
    primitives.append(primitives[-1])  
    return primitives

# ==== parse single tfrecord ====
def parse_single_example(example_proto):
    feature_description = {
        "episode_metadata/file_path": tf.io.FixedLenFeature([], tf.string),
        "steps/state": tf.io.VarLenFeature(tf.float32),  
    }
    return tf.io.parse_single_example(example_proto, feature_description)

# ==== summary container ====
all_move_texts = {}
all_states_list = []

# ==== main processing ====
for tfrecord_path in tqdm(tfrecord_files, desc="Reading TFRecord files"):
    raw_dataset = tf.data.TFRecordDataset(tfrecord_path)

    for raw_record in raw_dataset:
        try:
            example = parse_single_example(raw_record)

            file_path = example["episode_metadata/file_path"].numpy().decode()
            save_key = file_path

            state_seq = tf.sparse.to_dense(example["steps/state"]).numpy().astype(np.float64)

            if len(state_seq) == 0:
                print(f"[skip] {save_key} has no state")
                continue

            states = state_seq.reshape(-1, 33) 
            states_for_move = states[:, 28:31]   

            move_primitives = get_move_primitives_from_states(states_for_move)
            move_text_list = [move_text for move_text, move_vec in move_primitives]

            all_move_texts[save_key] = move_text_list
            all_states_list.append(states)

            print(f"[Done] {save_key}:{states.shape[0]} frames")
        except Exception as e:
            print(f"[Error] {tfrecord_path}: {e}")

# ==== save ====
all_states_concat = np.concatenate(all_states_list, axis=0)
np.save(os.path.join(save_dir, "all_states.npy"), all_states_concat)

with open(os.path.join(save_dir, "all_moves_list.json"), "w") as f:
    json.dump(all_move_texts, f, indent=2)

print("ALL Done（file_path -> move list）")


In [ ]:
#continue process move primitive 
import json
import os


INPUT_PATH = "./dexart/results_move_primitives/bucket/all_moves_list.json"
# middle output path
OUTPUT_PATH = "./dexart/results_move_primitives/bucket/raw_primitives.json" 
# final output folder
output_dir = "./dexart/results_move_primitives/bucket"

# load original moves data
with open(INPUT_PATH, "r") as f:
    raw_moves = json.load(f)

# build new format primitives.json
formatted_moves = {}

for episode_id_str, move_list in raw_moves.items():
    # build key: object_box_Task_{episode_id}_Demo_0
    formatted_key = f"{episode_id_str}"
    formatted_moves[formatted_key] = move_list

# save to target position
os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)
with open(OUTPUT_PATH, "w") as f:
    json.dump(formatted_moves, f)

print(f"Saved rawprimitives.json to {OUTPUT_PATH}")

# convert data for merge format
import json
import re

def convert_keys(input_path, output_path):
    with open(input_path, "r") as f:
        data = json.load(f)

    converted_data = {}
    for key, value in data.items():
        match = re.search(r"episode_(\d+)_combined\.npz$", key)


        if match:
            episode_id = match.group(1)
            converted_data[episode_id] = value
        else:
            print(f"[WARN] Key format not matched: {key}")

    with open(output_path, "w") as f:
        json.dump(converted_data, f, indent=2)
    print(f"Saved converted primitives file to {output_path}")


# ==== set path ====


convert_keys(OUTPUT_PATH, f"{output_dir}/primitives.json")

